In [ ]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [ ]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
ffK(s) = (@sprintf "%.5g" sd(3,s)/1e3)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe8(s) = (@sprintf "%.4g" sd(3,s)*1e8)

In [ ]:
@load "icc_test_small.jl.jld2"
dic_icc = copy(dic)

In [ ]:
uni_chimera_indices = findall(entry -> contains(entry, "uni_chimera(10000,"), dic["testName"])
wted_chimera_indices = findall(entry -> contains(entry, "wted_chimera(10000,"), dic["testName"])
uni_bndry_chimera_indices = findall(entry -> contains(entry, "uni_bndry_chimera(10000,"), dic["testName"])
wted_bndry_chimera_indices = findall(entry -> contains(entry, "wted_bndry_chimera(10000,"), dic["testName"])
suitesparse_indices = setdiff(1:length(dic["testName"]), vcat(uni_chimera_indices, wted_chimera_indices, uni_bndry_chimera_indices, wted_bndry_chimera_indices)),


In [ ]:
problem_idx = [suitesparse_indices, uni_chimera_indices, wted_chimera_indices, uni_bndry_chimera_indices, wted_bndry_chimera_indices]
problem_names = ["suitesparse", "uni_chimera", "wted_chimera", "uni_bndry_chimera", "wted_bndry_chimera"]

In [ ]:
ratios = []
errors = []
for indices in problem_idx
    ratio = dic["icc_tot"][indices] ./ dic["ne"][indices]
    push!(ratios, ratio)
    error = dic["icc_err"][indices]
    push!(errors, error)
end


In [ ]:
for (name, ratio, error) in zip(problem_names, ratios, errors)
    mean_val = mean(ratio)
    quantile_val = quantile(ratio, 0.75)
    max_val = maximum(ratio)
    
    @printf("%-20s - Ratio - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n", name, ffmicro(mean_val), ffmicro(quantile_val), ffmicro(max_val))
    mean_error = mean(error)
    quantile_error = quantile(error, 0.75)
    max_error = maximum(error)
    
    @printf("%-20s - Error - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n", name, ffe8(mean_error), ffe8(quantile_error), ffe8(max_error))
end